In [15]:
!pip install tensorflow tf-keras pandas matplotlib transformers


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [34]:
from tf_keras import Input
from transformers.pipelines.base import KeyDataset
import tensorflow as tf
from tensorflow.keras import layers
from transformers import ViTImageProcessor, TFViTForImageClassification, pipeline
import pandas as pd
import numpy as np
# Define paths
base_directory = './train/train'
test_directory = './test/'  # Ensure this directory contains the images to classify

# Prepare the dataset
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  base_directory,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width))
val_ds = tf.keras.utils.image_dataset_from_directory(
  base_directory,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
)


class_names = train_ds.class_names
print(class_names)

normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

for image_batch, labels_batch in train_ds:
  image_batch = processor(images=image_batch, return_tensors='tf')
  

image_batch, labels_batch = next(iter(train_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


num_classes = 5


model = TFViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

model.classifier = layers.Dense(5, activation='softmax')
print(model.layers)
model.compile()

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break
history = model.fit(train_ds, validation_data=val_ds, epochs=5)


Found 2026 files belonging to 5 classes.
Using 1621 files for training.
Found 2026 files belonging to 5 classes.
Using 405 files for validation.
['Cat', 'Cheetah', 'Leopard', 'Lion', 'Tiger']


2024-05-12 22:34:31.695586: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


7.0434508e-06 1.0


All PyTorch model weights were used when initializing TFViTForImageClassification.

All the weights of TFViTForImageClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


Model: "tf_vi_t_for_image_classification_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit (TFViTMainLayer)        multiple                  85798656  
                                                                 
 classifier (Dense)          multiple                  769000    
                                                                 
Total params: 86567656 (330.23 MB)
Trainable params: 86567656 (330.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
(32, 224, 224, 3)
(32,)
Epoch 1/5


2024-05-12 22:34:32.532772: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


ValueError: in user code:

    File "/home/germanua/Desktop/AI/dlchallenge/.venv/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/home/germanua/Desktop/AI/dlchallenge/.venv/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "/home/germanua/Desktop/AI/dlchallenge/.venv/lib/python3.11/site-packages/transformers/modeling_tf_utils.py", line 1672, in train_step  *
        y_pred = self(x, training=True)
    File "/home/germanua/Desktop/AI/dlchallenge/.venv/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 565, in error_handler  *
        del filtered_tb
    File "/home/germanua/Desktop/AI/dlchallenge/.venv/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 588, in __call__  *
        return super().__call__(*args, **kwargs)
    File "/home/germanua/Desktop/AI/dlchalleng

In [ ]:
print(model.layers[0].embeddings.patch_embeddings.projection)